Initialize the project directory and sub-folders if not done already.

Define functions that will be used after each code section to save checkpoints
so future runs don't start all over again.

In [ ]:
import pickle
from pathlib import Path

"""
Create essential directory structure for the project:
- fmri_project/: Main project directory
- fmri_project/checkpoints/: Store intermediate processing results
- fmri_project/results/: Store final analysis outputs
"""
project_dir = Path('/Users/daniilsergeyenko/PycharmProjects/fMRI_project')
checkpoints_dir = project_dir / 'checkpoints'
results_dir = project_dir / 'results'

# Create directories if they don't exist
for dir_path in [checkpoints_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)

def save_checkpoint(data, filename):
    """
    Save intermediate processing results to avoid unnecessary re-computation.

    Args:
        data: Any Python object to be saved
        filename: Name of the checkpoint file
    """
    filepath = checkpoints_dir / filename
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)
    print(f"Checkpoint saved: {filepath}")


def load_checkpoint(filename):
    """
    Load previously saved processing results.

    Args:
        filename: Name of the checkpoint file
    Returns:
        Loaded data if checkpoint exists, None otherwise
    """
    filepath = checkpoints_dir / filename
    if filepath.exists():
        with open(filepath, 'rb') as f:
            return pickle.load(f)
    return None

Create the events data frame based on the original data parameters and my unique research question.

In [ ]:
import pandas as pd

# Check if events DataFrame already exists
events = load_checkpoint('events.pkl')

if events is None:
    # The original data is sequenced as follows:
    # 'emotion': ['calm', 'afraid', 'delighted', 'depressed',
    #             'excited', 'delighted', 'depressed', 'calm',
    #             'excited', 'afraid']
    # Mapping instead to the groups I selected; neutral, positive, negative.
    # Then removing the redundant emotion column to avoid runtime UserWarning.

    events = pd.DataFrame({
        'onset': [0, 60, 120, 180, 240, 300, 360, 420, 480, 540],
        'duration': [30] * 10,
        'trial_type': ['neutral', 'negative', 'positive', 'negative',
                      'positive', 'positive', 'negative', 'neutral',
                      'positive', 'negative']
    })

    # Save events DataFrame
    save_checkpoint(events, 'events.pkl')
else:
    print("Loaded existing events DataFrame")

Run the single-subject analysis for each participant and save the results.

In [ ]:
from nilearn import image
from nilearn.glm.first_level import FirstLevelModel

# Check if subject results already exist
all_subject_contrasts = load_checkpoint('subject_contrasts.pkl')

if all_subject_contrasts is None:
    # I use open data of 40 participants.
    n_subjects = 40
    # Initialize lists to store results.
    all_subject_contrasts = []

    # Process each subject
    for subject_id in range(1, n_subjects + 1):
        subject_id_str = f'{subject_id:02d}'
        print(f'Processing subject {subject_id_str}...')

        # Check if individual subject results exist
        subject_results = load_checkpoint(f'subject_{subject_id_str}_results.pkl')

        if subject_results is None:
            # Load the subject's functional data
            func_file = f'ds005700/sub-{subject_id_str}/func/sub-{subject_id_str}_task-fe_bold.nii.gz'
            func_img = image.load_img(func_file)

            # Create and fit first-level model. With explicit parameters.
            model = FirstLevelModel(
                t_r=2.02697,  # Time(sec) it takes to scan one full head volume. written in each .json file.
                noise_model='ar1',
                standardize=True,
                hrf_model='spm',
                drift_model='cosine'
            )
            model.fit(func_img, events)

            # Define and compute contrasts
            contrasts = {
                'positive_vs_neutral': 'positive - neutral',
                'negative_vs_neutral': 'negative - neutral',
                'positive_vs_negative': 'positive - negative'
            }
            contrast_maps = {}
            for contrast_id, contrast_def in contrasts.items():
                contrast_maps[contrast_id] = model.compute_contrast(contrast_def)

            # Save individual subject results
            subject_results = {'contrasts': contrast_maps}
            save_checkpoint(subject_results, f'subject_{subject_id_str}_results.pkl')
        else:
            # in case the specific subject's results already exist fetch them instead.
            print(f"Loaded existing results for subject {subject_id_str}")
            contrast_maps = subject_results['contrasts']

        all_subject_contrasts.append(contrast_maps)

    # Save all subject results
    save_checkpoint(all_subject_contrasts, 'subject_contrasts.pkl')
else:
    print("Loaded existing subject results")

Run the group analysis, averaging all the participants and saving the results.

In [ ]:
import numpy as np
from nilearn.glm.second_level import SecondLevelModel
from nilearn import plotting, image

# Check if group results already exist
group_results = load_checkpoint('group_results.pkl')

if group_results is None:
    # Create design matrix for second-level analysis
    n_subjects = 40
    design_matrix = pd.DataFrame({
        'intercept': np.ones(n_subjects),
    })

    # Initialize second-level model with explicit parameters
    second_level_model = SecondLevelModel(
        smoothing_fwhm=8.0,  # Add explicit smoothing
        n_jobs=1,  # Ensure single job for stability
        memory=None,  # Don't cache computations
        verbose=0  # Reduce verbosity
    )

    # Define contrasts to analyze - same as within each subject.
    contrasts_to_analyze = [
        'positive_vs_neutral',
        'negative_vs_neutral',
        'positive_vs_negative'
    ]

    # Initialize dictionary to store results for each of the three contrasts.
    group_results = {}

    # Process each contrast
    for contrast_name in contrasts_to_analyze:
        print(f"Running group analysis for {contrast_name}...")

        # Extract and preprocess contrast maps for all subjects
        contrast_maps_list = []

        # Get the first map to use as reference
        first_map = all_subject_contrasts[0][contrast_name]
        reference_affine = first_map.affine
        reference_shape = first_map.shape[:3]  # Exclude time dimension if present

        # Process each subject's contrast map
        for subject_contrasts in all_subject_contrasts:
            contrast_img = subject_contrasts[contrast_name]

            # Ensure 3D
            if len(contrast_img.shape) == 4:
                contrast_img = image.index_img(contrast_img, 0)

            # Resample to match reference with explicit parameters
            resampled_img = image.resample_img(
                contrast_img,
                target_affine=reference_affine,
                target_shape=reference_shape,
                interpolation='continuous',
                force_resample=True,
                copy_header=True,
                clip=True
            )

            contrast_maps_list.append(resampled_img)

        # Attempt to run the second_level_model, utilizing try;except in case the previous processes weren't saved correctly.
        try:
            # Fit the second-level model
            second_level_model.fit(contrast_maps_list, design_matrix=design_matrix)

            # Compute the group-level contrast
            z_map = second_level_model.compute_contrast(
                output_type='z_score',
                second_level_stat_type='t'  # Explicitly specify t-test
            )
            p_map = second_level_model.compute_contrast(
                output_type='p_value',
                second_level_stat_type='t'
            )

            # Store results
            group_results[contrast_name] = (z_map, p_map)

        except Exception as e:
            print(f"Error processing contrast {contrast_name}: {str(e)}")
            continue

    # Save group results
    save_checkpoint(group_results, 'group_results.pkl')
else:
    print("Loaded existing group results")

Plotting the results of all contrast comparisons.

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import norm

# Plotting parameters
z_threshold = 3.1  # Corresponds to p < 0.001
p_val = norm.sf(z_threshold)  # Convert Z-score to p-value
print(f"Z-score threshold {z_threshold} corresponds to p-value < {p_val:.3e}")

# Plot results for each contrast
for contrast_name, (z_map, p_map) in group_results.items():
    print(f"Plotting results for {contrast_name}...")

    # Create figure with multiple views
    fig = plt.figure(figsize=(15, 5))

    # Plot sagittal, coronal, and axial views with statistical threshold
    display = plotting.plot_stat_map(
        z_map,
        threshold=z_threshold,  # Apply threshold only during visualization
        display_mode='ortho',
        title=f'Group-level {contrast_name}\n'
              f'(threshold: z>{z_threshold}, p<{p_val:.3e})',
        figure=fig
    )

    # Save the figure
    plt.savefig(f'results/group_analysis_{contrast_name}(Z scores).png',
                dpi=300,
                bbox_inches='tight')
    plt.close()